## Building a DCGAN from Scratch

The first order of the day is to build the generator and the discriminator.


In [1]:
import torch
import torch.nn as nn


In [2]:
z = torch.rand(50,1,100)

Here we build the DCGAN generator from the LSUN generator

In [11]:
class DCGAN_generator (nn.Module):
    def __init__(self,z):
        super().__init__()
        # Determining the batch size
        self.batch_size = z.shape[0]

        self.projection = nn.Linear(100,16384)
        self.conv_1 = nn.Sequential(nn.ConvTranspose2d(in_channels=1024,out_channels=512,kernel_size=2,stride=2,padding=0),nn.BatchNorm2d(512),nn.ReLU())
        self.conv_2 = nn.Sequential(nn.ConvTranspose2d(in_channels=512,out_channels=256,kernel_size=2,stride=2,padding=0),nn.BatchNorm2d(256),nn.ReLU())
        self.conv_3 = nn.Sequential(nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=2,stride=2,padding=0),nn.BatchNorm2d(128),nn.ReLU())
        self.conv_4 = nn.Sequential(nn.ConvTranspose2d(in_channels=128,out_channels=3,kernel_size=2,stride=2,padding=0),nn.Tanh())
     
        


    
    def forward(self, x):
        
    
        
        x = self.projection(x)
        print(self.batch_size)
        x = x.view(self.batch_size,1024,4,4)
       
        

        x = self.conv_1(x)
        
        x = self.conv_2(x)
        x = self.conv_3(x) 
        x = self.conv_4(x)
        

        return x   


In [5]:
model = DCGAN_generator(z)
output = model(z)


50


Also did the LSUN discriminator

In [6]:
class DCGAN_discriminator(nn.Module):
    def __init__(self,z):
        super().__init__()
        self.batch_size = z.shape[0]
        
        self.conv_1 = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=128,kernel_size=2,stride=2),nn.LeakyReLU())
        self.conv_2 = nn.Sequential(nn.Conv2d(in_channels=128,out_channels=256,kernel_size=2,stride=2),nn.BatchNorm2d(256),nn.LeakyReLU())
        self.conv_3 = nn.Sequential(nn.Conv2d(in_channels=256,out_channels=512,kernel_size=2,stride=2),nn.BatchNorm2d(512),nn.LeakyReLU())
        self.conv_4 = nn.Sequential(nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=2,stride=2),nn.BatchNorm2d(1024),nn.LeakyReLU())
        self.flatten = nn.Flatten(0,3)
        self.fc1 = nn.Linear(16384*self.batch_size,1*self.batch_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        
        
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.sigmoid(x)
        

        return x
        


In [7]:
class DCGAN(nn.Module):
    def __init__(self,z):
        super().__init__()
        self.z = z
        self.generated_output = DCGAN_generator(z)
        

    def forward(self,z):
        output = self.generated_output(z)
        model = DCGAN_discriminator(output)
        output = model(output)
        return output




In [10]:
model = DCGAN(z)
output = model(z)
print(output)


50
tensor([0.4403, 0.3142, 0.4655, 0.4392, 0.3537, 0.4460, 0.3353, 0.4553, 0.4852,
        0.4968, 0.3494, 0.5184, 0.5660, 0.4248, 0.4656, 0.4797, 0.4389, 0.4486,
        0.4599, 0.4747, 0.5460, 0.6025, 0.3353, 0.5050, 0.5015, 0.3402, 0.4296,
        0.3742, 0.6848, 0.5963, 0.4113, 0.4749, 0.6757, 0.4915, 0.4695, 0.4984,
        0.4246, 0.5147, 0.4786, 0.3943, 0.4062, 0.4584, 0.4969, 0.5189, 0.3490,
        0.5025, 0.7390, 0.4787, 0.4784, 0.4570], grad_fn=<SigmoidBackward0>)
